In [1]:
!pip install --upgrade openai


In [2]:
!pip install nest_asyncio


In [14]:
import os
import json
import time
from openai import OpenAI
from dotenv import load_dotenv

# .env 파일에서 API 키를 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=api_key)

# Fine-tuning 및 예측 프로세스를 실행하는 함수 정의
def main():
    # 훈련 데이터를 준비하고 .jsonl 파일로 저장
    data = [
        {"messages": [{"role": "system", "content": "에스카노르처럼 답변해줘."}, {"role": "user", "content": "낮의 에스카노르는 어떤 성격인가요?"}], "completion": "낮의 에스카노르는 오만하고 자신감 넘치며 강력합니다."},
        {"messages": [{"role": "system", "content": "에스카노르처럼 답변해줘."}, {"role": "user", "content": "밤의 에스카노르는 어떤 성격인가요?"}], "completion": "밤에는 에스카노르가 소심해지고 공손하게 말합니다."},
        # 필요에 따라 데이터를 추가하세요...
    ]

    # 데이터를 .jsonl 파일로 저장
    file_path = "fine_tune_data.jsonl"
    with open(file_path, "w", encoding="utf-8") as f:
        for entry in data:
            json.dump(entry, f, ensure_ascii=False)
            f.write("\n")

    # 파일을 동기식으로 업로드하고 파일 ID를 가져옴
    with open(file_path, "rb") as file:
        upload_response = client.files.create(file=file, purpose="fine-tune")  # 동기식 처리
    
    # upload_response는 FileObject 객체이므로, .id 속성으로 접근
    file_id = upload_response.id  # FileObject에서 id 속성으로 접근
    print(f"파일이 성공적으로 업로드되었습니다. 파일 ID: {file_id}")

    # Fine-tuning 작업 시작
    fine_tune_response = client.finetune.create(training_file=file_id, model="gpt-4-mini")  # Fine-tune 생성
    fine_tune_job_id = fine_tune_response["id"]
    print(f"Fine-tuning 작업이 시작되었습니다. 작업 ID: {fine_tune_job_id}")

    # Fine-tuning 상태를 주기적으로 확인하며 완료될 때까지 대기
    while True:
        status_response = client.finetune.retrieve(id=fine_tune_job_id)  # 상태 확인
        status = status_response["status"]
        print(f"Fine-tuning 작업 상태: {status}")
        if status == "succeeded":
            break
        elif status == "failed":
            raise Exception("Fine-tuning 작업이 실패했습니다.")
        time.sleep(10)  # 동기식으로 대기

    # Fine-tuning 완료 후 모델 이름 확인
    fine_tuned_model = status_response["fine_tuned_model"]
    print(f"Fine-tuning 완료! 새 모델 이름: {fine_tuned_model}")

    # Fine-tuned 모델로 예측 수행
    response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=[{"role": "user", "content": "낮의 에스카노르는 어떤 성격인가요?"}]
    )

    print(response["choices"][0]["message"]["content"])

# main() 함수 실행
main()


파일이 성공적으로 업로드되었습니다. 파일 ID: file-1cFpDdgzV8EMFfKORdQqkJIn


AttributeError: 'OpenAI' object has no attribute 'finetune'